<a href="https://colab.research.google.com/github/SimeonHristov99/AI_PredictChurn_Adaboost/blob/main/solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Description: This notebook performs data cleaning, feature engineering and classification using Adaboost in order to predict the churn rate for the "churn rate" dataset.

In [416]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import numpy as np

from sklearn.model_selection import train_test_split

In [417]:
df = pd.read_csv('/content/training_data.csv') # Assuming it is preloaded in Google Colab
df.head()

,tenure,age,address,income,ed,employ,equip,callcard,wireless,longmon,tollmon,equipmon,cardmon,wiremon,longten,tollten,cardten,voice,pager,internet,callwait,confer,ebill,loglong,logtoll,lninc,custcat,churn
0,11.0,33.0,7.0,136.0,5.0,5.0,0.0,1.0,1.0,4.40,20.75,0.0,15.25,35.7,42.00,211.45,125.0,1.0,1.0,0.0,1.0,1.0,0.0,1.482,3.033,4.913,4.0,1.0
1,33.0,33.0,12.0,33.0,2.0,0.0,0.0,0.0,0.0,9.45,0.00,0.0,0.00,0.0,288.80,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.246,3.240,3.497,1.0,1.0
2,23.0,30.0,9.0,30.0,1.0,2.0,0.0,0.0,0.0,6.30,0.00,0.0,0.00,0.0,157.05,0.00,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.841,3.240,3.401,3.0,0.0
3,38.0,35.0,5.0,76.0,2.0,10.0,1.0,1.0,1.0,6.05,45.00,50.1,23.25,64.9,239.55,1873.05,880.0,1.0,1.0,1.0,1.0,1.0,1.0,1.800,3.807,4.331,4.0,0.0
4,7.0,35.0,14.0,80.0,2.0,15.0,0.0,1.0,0.0,7.10,22.00,0.0,23.75,0.0,47.45,166.10,145.0,1.0,0.0,0.0,1.0,1.0,0.0,1.960,3.091,4.382,3.0,0.0


In [418]:
df.shape

(200, 28)

In [419]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 28 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tenure    200 non-null    float64
 1   age       200 non-null    float64
 2   address   200 non-null    float64
 3   income    200 non-null    float64
 4   ed        200 non-null    float64
 5   employ    200 non-null    float64
 6   equip     200 non-null    float64
 7   callcard  200 non-null    float64
 8   wireless  200 non-null    float64
 9   longmon   200 non-null    float64
 10  tollmon   200 non-null    float64
 11  equipmon  200 non-null    float64
 12  cardmon   200 non-null    float64
 13  wiremon   200 non-null    float64
 14  longten   200 non-null    float64
 15  tollten   200 non-null    float64
 16  cardten   200 non-null    float64
 17  voice     200 non-null    float64
 18  pager     200 non-null    float64
 19  internet  200 non-null    float64
 20  callwait  200 non-null    float6

In [420]:
df.describe()

,tenure,age,address,income,ed,employ,equip,callcard,wireless,longmon,tollmon,equipmon,cardmon,wiremon,longten,tollten,cardten,voice,pager,internet,callwait,confer,ebill,loglong,logtoll,lninc,custcat,churn
count,200.000000,200.000000,200.000000,200.000000,200.00000,200.00000,200.000000,200.000000,200.000000,200.00000,200.000000,200.000000,200.000000,200.0000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.00000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,35.505000,41.165000,11.650000,75.130000,2.82500,10.22500,0.425000,0.705000,0.290000,11.78925,13.240000,15.783250,14.358750,12.2170,577.770750,507.007500,650.663750,0.295000,0.275000,0.440000,0.45500,0.460000,0.440000,2.193285,3.229185,3.951015,2.475000,0.290000
std,21.640971,13.076803,10.158419,128.430468,1.28555,8.95743,0.495584,0.457187,0.454901,9.88725,15.643842,19.557103,14.781269,21.6326,746.608653,820.027858,941.470689,0.457187,0.447635,0.497633,0.49922,0.499648,0.497633,0.731282,0.281019,0.752553,1.079445,0.454901
min,1.000000,19.000000,0.000000,9.000000,1.00000,0.00000,0.000000,0.000000,0.000000,1.10000,0.000000,0.000000,0.000000,0.0000,1.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.095000,1.749000,2.197000,1.000000,0.000000
25%,16.750000,31.000000,3.000000,31.000000,2.00000,3.00000,0.000000,0.000000,0.000000,5.53750,0.000000,0.000000,0.000000,0.0000,79.337500,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,1.711750,3.226500,3.434000,2.000000,0.000000
50%,33.500000,40.000000,9.000000,48.000000,3.00000,7.50000,0.000000,1.000000,0.000000,8.25000,0.000000,0.000000,12.500000,0.0000,289.525000,0.000000,342.500000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,2.110000,3.240000,3.871000,2.000000,0.000000
75%,55.250000,51.000000,18.000000,80.000000,4.00000,17.00000,1.000000,1.000000,1.000000,14.30000,24.750000,33.012500,20.750000,23.4625,806.762500,724.162500,921.250000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,2.660000,3.240000,4.382000,3.000000,1.000000
max,72.000000,76.000000,48.000000,1668.000000,5.00000,44.00000,1.000000,1.000000,1.000000,62.30000,68.500000,63.250000,109.250000,109.7000,4333.000000,4938.600000,7515.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,4.132000,4.227000,7.419000,4.000000,1.000000


In [421]:
class DecisionStump:

  def __init__(self):
    self.polarity = 1 # whether a sample is positive or negative. Used since if the error is flipped, the polarity should also be flipped
    self.feature_idx = None
    self.threshold = None
    self.alpha = None
  
  # Look at only one feature for each sample. Compare it to self.threshold. If it's lower then -1, else +1.
  def predict(self, X):
    num_samples = X.shape[0]
    feature_column = X[:, self.feature_idx] # Get only the required feature

    predictions = np.ones(num_samples)
    if self.polarity == 1:
      predictions[feature_column < self.threshold] = -1
    else:
      predictions[feature_column > self.threshold] = -1
    
    return predictions

class Adaboost:

    def __init__(self, num_classifiers=5):
      self.num_classifiers = num_classifiers
    
    def fit(self, X, y):
      num_samples, num_features = X.shape

      # generate an array with size "num_samples" and with values 1/num_samples
      W = np.full(num_samples, (1/num_samples))

      self.classifiers = []
      for _ in range(self.num_classifiers):
        classifier = DecisionStump()

        # find the best feature, i.e. with highest information gain/entropy
        min_error = float('inf')
        
        for feature_idx in range(num_features):
          feature_column = X[:, feature_idx]
          thresholds = np.unique(feature_column)
        
          for threshold in thresholds:
            polarity = 1
            predictions = np.ones(num_samples)
            predictions[feature_column < threshold] = -1

            misclassified = W[y != predictions]
            err = np.sum(misclassified)

            if err > 0.5:
              err = 1 - err
              polarity = -1
            
            if err < min_error:
              min_error = err
              classifier.polarity = polarity
              classifier.threshold = threshold
              classifier.feature_idx = feature_idx

        EPS = 1e-10 # in order not to (accidentally) divide by 0
        classifier.alpha = 0.5 * np.log((1 - min_error) / (min_error + EPS))

        predictions = clf.predict(X)

        # update weights
        W *= np.exp(-classifier.alpha * y * predictions)
        W /= np.sum(W)

        self.classifiers.append(classifier)
    
    def predict(self, X):
      classifier_predictions = [clf.alpha * clf.predict(X) for clf in self.classifiers]
      y_pred = np.sum(classifier_predictions, axis=0)
      return np.sign(y_pred)

In [422]:
# # Decision stump used as weak classifier
# class DecisionStump():
#     def __init__(self):
#         self.polarity = 1
#         self.feature_idx = None
#         self.threshold = None
#         self.alpha = None

#     def predict(self, X):
#         n_samples = X.shape[0]
#         X_column = X[:, self.feature_idx]
#         predictions = np.ones(n_samples)
#         if self.polarity == 1:
#             predictions[X_column < self.threshold] = -1
#         else:
#             predictions[X_column > self.threshold] = -1

#         return predictions


# class Adaboost():

#     def __init__(self, n_clf=5):
#         self.n_clf = n_clf

#     def fit(self, X, y):
#         n_samples, n_features = X.shape

#         # Initialize weights to 1/N
#         w = np.full(n_samples, (1 / n_samples))

#         self.clfs = []
#         # Iterate through classifiers
#         for _ in range(self.n_clf):
#             clf = DecisionStump()

#             min_error = float('inf')
#             # greedy search to find best threshold and feature
#             for feature_i in range(n_features):
#                 X_column = X[:, feature_i]
#                 thresholds = np.unique(X_column)

#                 for threshold in thresholds:
#                     # predict with polarity 1
#                     p = 1
#                     predictions = np.ones(n_samples)
#                     predictions[X_column < threshold] = -1

#                     # Error = sum of weights of misclassified samples
#                     misclassified = w[y != predictions]
#                     error = sum(misclassified)

#                     if error > 0.5:
#                         error = 1 - error
#                         p = -1

#                     # store the best configuration
#                     if error < min_error:
#                         clf.polarity = p
#                         clf.threshold = threshold
#                         clf.feature_idx = feature_i
#                         min_error = error

#             # calculate alpha
#             EPS = 1e-10
#             clf.alpha = 0.5 * np.log((1.0 - min_error + EPS) / (min_error + EPS))

#             # calculate predictions and update weights
#             predictions = clf.predict(X)

#             w *= np.exp(-clf.alpha * y * predictions)
#             # Normalize to one
#             w /= np.sum(w)

#             # Save classifier
#             self.clfs.append(clf)

#     def predict(self, X):
#         clf_preds = [clf.alpha * clf.predict(X) for clf in self.clfs]
#         y_pred = np.sum(clf_preds, axis=0)
#         y_pred = np.sign(y_pred)

#         return y_pred

In [423]:
def accuracy(y_true, y_pred):
  return np.sum(y_true == y_pred) / len(y_true)

In [424]:
model = Adaboost(5) # using 5 classifiers

X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 0:27], df.iloc[:, -1])

type(X_train.iloc[:, 0].to_numpy())

model.fit(X_train.to_numpy(),y_train.to_numpy())

y_pred = model.predict(X_test.to_numpy())
print(f"Accuracy for: {accuracy(y_test, y_pred)}")

Accuracy for: 0.28


In [425]:
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier(n_estimators=5)
clf.fit(X_train, y_train)
accuracy(y_test, clf.predict(X_test))

0.62